# Mosquito Data Analysis and Predictive Modeling for West Nile Virus in Chicago

## Introduction
This project involves a comprehensive analysis and predictive modeling of mosquito data provided by the City of Chicago, spanning from 2008 to 2019. In the first part of the project, we utilize `pandas`, `matplotlib`, and `seaborn` to perform detailed data analysis and create visualizations to understand mosquito trends in the greater Chicago area. The second part focuses on developing predictive models: a linear regression model to predict mosquito numbers and a logistic regression model to estimate the prevalence of the West Nile Virus. These models aim to provide valuable insights and tools for effective mosquito control and public health strategies.